In [19]:
# Install all the necessary python libraries if anaconda is not able to find them.

In [20]:
# Import all the necessary python libraries
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
from pprint import pprint

In [21]:
# Define CONSTANTS
FNB_IMAGE_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/button/img'
FNB_OPENING_HOURS_BTN_DOWN_XPATH = '//*[@jsaction[starts-with(., "pane.openhours")]]' #and @role="button" and @tabindex="0" and @aria-expanded="false"]'
FNB_OPENING_HOURS_BTN_RIGHT_XPATH = '//*[@data-item-id="oh"]'
FNB_OPENING_HOURS_DETAILS_XPATH = '//*[@aria-label[contains(., "Hide open hours for the week")]]'
FNB_OPENING_HOURS_BACK_BTN_XPATH = '//*[@aria-label="Back"]'
FNB_OPENING_HOURS_CLOSED_XPATH = '//span[@class=" fCEvvc"]' #Temporarily Closed / #Permanently Closed
FNB_REVIEWS_TAB_XPATH = '//button[@role="tab" and @aria-label[starts-with(., "Reviews")]]'
FNB_SORT_REVIEWS_BTN_XPATH = '//button[@aria-label="Sort reviews"]'
FNB_SORT_BY_NEWEST_BTN_XPATH = '//div[@class="fxNQSd" and @data-index="1" and @role="menuitemradio"]'
FNB_REVIEW_LIST_XPATH = '//div[@class="jftiEf fontBodyMedium "]'
FNB_REVIEW_SCROLLABLE_CONTENT_XPATH = '//div[@class="m6QErb DxyBCb kA9KIf dS8AEf "]'
FNB_REVIEW_COUNT_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span'#'//span[contains(@aria-label, "review")]'
FNB_REVIEW_SEE_MORE_XPATH = './/button[@aria-label="See more"]'
FNB_REVIEW_RATING_XPATH = './/span[@class="kvMYJc" and @role="img"]'
FNB_REVIEW_CONTENT_XPATH = './/span[@class="wiI7pd"]'
FNB_REVIEW_PHOTOS_XPATH = './/button[@class="Tya61d"]'
FNB_REVIEWER_XPATH = './/button[@class="al6Kxe"]'
FNB_REVIEWER_NAME_XPATH = './/div[@class="d4r55 "]'
FNB_REVIEWER_DETAILS_XPATH = './/div[@class="RfnDt "]'
FNB_CLOSED_XPATH = '//span[@class=" fCEvvc"]'

In [22]:
# Initialize Variables
review_list = []

In [23]:
file_path = 'fnb_list_part_2_cleaned.csv'
df_final = pd.read_csv(file_path, encoding='utf-8')
df_final['Reviews'] = None
df_final.head(10)


,Name,Simplified Name,Category,Area,Located In,Address,Phone,Google Map Url,Menu Url,Website Url,Order Url,Reviews
0,#leleh,NaN,['Cafe'],Cheras,MyTOWN Shopping Centre,"Seksyen, 90, Jalan Cochrane, Cheras, 55100 Kua...",03-2720 1772,https://www.google.com/maps/place/%23leleh/dat...,NaN,NaN,NaN,None
1,1 Plus 1 Coffee • Fruit ( Café ),1 Plus 1 Coffee,['Cafe'],Puchong,NaN,"1, Jalan Puteri 7/11, Bandar Puteri, 47100 Puc...",016-934 7388,https://www.google.com/maps/place/1+Plus+1+Cof...,NaN,http://web.facebook.com/profile.pho?id=1000844...,https://grab.onelink.me/2695613898?pid=inappsh...,None
2,10 Pasento Puchong,10 Pasento Puchong,['Cafe'],Puchong,Floor 1 · Hotel Sri Sutra,"1st Floor, 10, Jalan Kenari 6, Bandar Puchong ...",NaN,https://www.google.com/maps/place/10+Pasento+P...,NaN,https://www.instagram.com/10_pasento/,NaN,None
3,103 Coffee,103 Coffee,['Cafe'],Seri Kembangan,NaN,"103-G, Jalan Radin Bagus, Bandar Baru Sri Peta...",012-519 7103,https://www.google.com/maps/place/103+Coffee/d...,http://103coffeeworkshop.beepit.com/,https://103coffee.com/,NaN,None
4,128 Thai,128 Thai,['Western Food'],Subang Jaya,NaN,"42, Jalan SS 15/4c, Ss 15, 47500 Subang Jaya, ...",016-233 0609,https://www.google.com/maps/place/128+Thai/dat...,NaN,https://128thai.business.site/?m=true,https://wa.me/message/EY3JJ7HNEGQ4G1,None
5,"1475 Food Venture- Burgers, Western and more",1475 Food Venture- Burgers,['Fast Food'],Petaling Jaya,1 Utama Shopping Centre,"Lot LG221, Lower Ground Floor, Promenade, 1 Ut...",03-7726 2361,https://www.google.com/maps/place/1475+Food+Ve...,NaN,https://www.mcdonalds.com.my/,https://www.foodpanda.my/restaurant/m3oj/mcdon...,None
6,16 degrees coffee & grocer,16 degrees coffee & grocer,['Cafe'],Subang Jaya,NaN,"9, Jalan USJ 16/2g, UEP SUBANG JAYA, 47630 Sub...",011-1148 8796,https://www.google.com/maps/place/16+degrees+c...,NaN,https://16degrees.shopzpin.com/#/outlet-produc...,NaN,None
7,17 Eleven Roof Cafe,17 Eleven Roof Cafe,['Cafe'],Seri Kembangan,NaN,"20-1a, Jalan Equine 1B, Taman Equine, 43300 Se...",017-393 6456,https://www.google.com/maps/place/17+Eleven+Ro...,NaN,https://instagram.com/17eleven_roofcafe?igshid...,NaN,None
8,17 Hills Restaurant,17 Hills Restaurant,NaN,Petaling Jaya,NaN,"22, Jalan 17/54, Seksyen 17, 46400 Petaling Ja...",03-7931 3307,https://www.google.com/maps/place/17+Hills+Res...,https://www.facebook.com/17HillsRestaurant/menu,https://www.instagram.com/17hills.pj/,NaN,None
9,17gram beans,17gram beans,['Cafe'],Subang Jaya,Subang Jaya Medical Centre (SJMC),"Lot SJB1.009-011, B1 Escalator Lobby Subang Ja...",011-5859 4594,https://www.google.com/maps/place/17gram+beans...,NaN,http://obriens.com.my/,https://www.foodpanda.my/restaurant/xiyh/obrie...,None


In [24]:
def convert_opening_hours_to_dict(opening_hours):

    day_parts = opening_hours.replace('\u202f', ' ').replace('. Hide open hours for the week', '').split(';')

    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    open_hours_dict = {
        part.split(',')[0].strip(): part.split(',', 1)[1].strip()
        for part in day_parts
    }

    sorted_open_hours_dict = {day: open_hours_dict[day] for day in days_order if day in open_hours_dict}

    return sorted_open_hours_dict


In [25]:
def parse_count_text(count):
    return int(count.replace(",", ""))

In [26]:
def parse_reviewer_details(reviewer_details):
    
    parts = [part.strip() for part in reviewer_details.split("·")]
    
    reviewer_type = None
    reviewer_reviews_count = 0
    reviewer_photos_count = None

    for part in parts:
        if 'Local Guide' in part:
            reviewer_type = 'Local Guide'
        elif 'review' in part:
            reviewer_reviews_count = parse_count_text(part.split()[0])
        elif 'photo' in part:
            reviewer_photos_count = parse_count_text(part.split()[0])
    
    return reviewer_type, reviewer_reviews_count, reviewer_photos_count

In [27]:
driver = webdriver.Safari()
driver.maximize_window()

for index, row in tqdm(df_final.iterrows(), desc="Processing rows"):
    url = row['Google Map Url']
    driver.get(url)
    fnb_image_element = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, FNB_IMAGE_XPATH))
    )
    fnb_image_url = fnb_image_element.get_attribute('src')
    #tqdm.write(f'{index+1}. {fnb_image_url}')
    time.sleep(1)
    total_reviews_count = 0
    current_reviews_count = 0
    review_list = []

    try:
        review_count_element = driver.find_element(By.XPATH, FNB_REVIEW_COUNT_XPATH)
        if review_count_element:
            total_reviews_count = parse_count_text(review_count_element.text.split("(")[1].split(")")[0])
            
    except:
        total_reviews_count = 0

    try:
        # CASE 1: OPENING HOURS BUTTON IS POINTING DOWN
        opening_hours_btn_elem = driver.find_element(By.XPATH, FNB_OPENING_HOURS_BTN_DOWN_XPATH)
        time.sleep(1)
        try:
            opening_hours_btn_elem.click()
        except:
            opening_hours_btn_elem.send_keys(Keys.ENTER)
        time.sleep(1)
        opening_hours_elem = driver.find_element(By.XPATH, FNB_OPENING_HOURS_DETAILS_XPATH)
        opening_hours = opening_hours_elem.get_attribute('aria-label')
        opening_hours_dict = convert_opening_hours_to_dict(opening_hours)

    except NoSuchElementException:

        # CASE 2: OPENING HOURS BUTTON IS POINTING RIGHT
        try:
            opening_hours_btn_elem = driver.find_element(By.XPATH, FNB_OPENING_HOURS_BTN_RIGHT_XPATH)
            time.sleep(1)
            try:
                opening_hours_btn_elem.click()
            except:
                opening_hours_btn_elem.send_keys(Keys.ENTER)
            time.sleep(1)
            opening_hours_elem = driver.find_element(By.XPATH, FNB_OPENING_HOURS_DETAILS_XPATH)
            opening_hours = opening_hours_elem.get_attribute('aria-label')
            opening_hours_dict = convert_opening_hours_to_dict(opening_hours)
            time.sleep(0.5)
            driver.find_element(By.XPATH, FNB_OPENING_HOURS_BACK_BTN_XPATH).click()
            time.sleep(1)

        except NoSuchElementException:

            # CASE 3: FNB IS TEMPORARILY CLOSED OR PERMANENTLY CLOSED
            try:
                opening_hours_dict = driver.find_element(By.XPATH, FNB_OPENING_HOURS_CLOSED_XPATH).text

            except NoSuchElementException:
                
                # CASE 4: NO OPENING HOURS STATED
                opening_hours_dict = None
            time.sleep(1)
        pass

    #tqdm.write(f'{index+1}. {opening_hours_dict}')

    if total_reviews_count != 0:
        driver.find_element(By.XPATH, FNB_REVIEWS_TAB_XPATH).click()
        time.sleep(1)

        driver.find_element(By.XPATH, FNB_SORT_REVIEWS_BTN_XPATH).click()
        time.sleep(1)
        
        driver.find_element(By.XPATH, FNB_SORT_BY_NEWEST_BTN_XPATH).click()
        time.sleep(1)

        while True:

            scrollable_review_content_element = driver.find_element(By.XPATH, FNB_REVIEW_SCROLLABLE_CONTENT_XPATH)
            review_list_element = driver.find_elements(By.XPATH, FNB_REVIEW_LIST_XPATH)
            time.sleep(0.5)
            current_reviews_count = len(review_list_element)
            driver.execute_script(
                'arguments[0].scrollTop = arguments[0].scrollHeight', 
                scrollable_review_content_element
            )

            if current_reviews_count >= 100:
                break

            elif current_reviews_count == total_reviews_count:
                break
        
        for review in review_list_element:
        #review = review_list_element[14]
            review_rating = 0
            review_text = None
            review_photos = []
            reviewer_name = ''
            reviewer_url = ''
            reviewer_type = None
            reviewer_reviews_count = 0
            reviewer_photos_count = None
            
            reviewer_element = review.find_element(By.XPATH, FNB_REVIEWER_XPATH)

            retry_count = 0
            while retry_count < 2:
                try:
                    reviewer_url = reviewer_element.get_attribute("data-href")
                    time.sleep(0.5)

                    reviewer_name = reviewer_element.find_element(By.XPATH, FNB_REVIEWER_NAME_XPATH).text
                    time.sleep(0.5)

                    reviewer_details = reviewer_element.find_element(By.XPATH, FNB_REVIEWER_DETAILS_XPATH).text
                    time.sleep(0.5)

                    reviewer_type, reviewer_reviews_count, reviewer_photos_count = parse_reviewer_details(reviewer_details)
                    time.sleep(0.5)

                    review_rating_element = review.find_element(By.XPATH, FNB_REVIEW_RATING_XPATH)
                    review_rating = int(review_rating_element.get_attribute('aria-label').split()[0])
                    time.sleep(0.5)

                    break
                except:
                    retry_count = retry_count + 1
            try:
                see_more_btn = review.find_element(By.XPATH, FNB_REVIEW_SEE_MORE_XPATH)
                if see_more_btn:
                    see_more_btn.click()
                    time.sleep(0.5)
            except:
                see_more_btn = None
            
            try:
                review_content_element = review.find_element(By.XPATH, FNB_REVIEW_CONTENT_XPATH)
                if review_content_element:
                    review_text = review_content_element.text
                    time.sleep(0.5)
            except:
                review_text = None
            
            try:
                review_photos_element = review.find_elements(By.XPATH, FNB_REVIEW_PHOTOS_XPATH)
                time.sleep(0.5)
                if review_photos_element:
                    for photos in review_photos_element:
                        photo_style_attribute = photos.get_attribute('style')
                        url_match = re.search(r'url\("([^"]+)"\)', photo_style_attribute)
                        photo_url = url_match.group(1)
                        review_photos.append(photo_url)
            except:
                review_photos = None
        
            review = {
                "Rating": review_rating,
                "Text": review_text,
                "Photos": review_photos,
                "Reviewer": {
                    "Name": reviewer_name,
                    "Reviews Count": reviewer_reviews_count,
                    "Photos Count": reviewer_photos_count,
                    "Type": reviewer_type,
                    "Url": reviewer_url
                }
            }#
            review_list.append(review)
        df_final.at[index, 'Reviews'] = review_list
        df_final.to_csv('fnb_list_part_5_reviews.csv', index=False)

Processing rows: 0it [00:00, ?it/s]

{'Rating': 2, 'Text': 'Less satisfactory', 'Photos': [], 'Reviewer': {'Name': 'singgih gayuh', 'Reviews Count': 2, 'Photos Count': None, 'Type': None, 'Url': 'https://www.google.com/maps/contrib/115700974126754522069/reviews?hl=en'}}
{'Rating': 3, 'Text': 'Water - 5 stars\nFood - 3 stars\n\nKuetiau is delicious when eaten with pickled chilies, not with pickled chilies. The price is terrible too. Umm.', 'Photos': ['https://lh5.googleusercontent.com/p/AF1QipPHcww8HPr9itIJtF0P0HiRazMMh5xBX55Z9Qrm=w600-h900-p-k-no', 'https://lh5.googleusercontent.com/p/AF1QipMGVT3LgJ9yu1FZEbsM5TvZFQo3quTcJCsXc-0Z=w600-h900-p-k-no'], 'Reviewer': {'Name': 'Nuryy', 'Reviews Count': 26, 'Photos Count': 64, 'Type': 'Local Guide', 'Url': 'https://www.google.com/maps/contrib/110861165917347919290/reviews?hl=en'}}
{'Rating': 1, 'Text': 'Worst fried kuey teow & char kuey teow ever. Not even deserve 1 star', 'Photos': [], 'Reviewer': {'Name': 'tira', 'Reviews Count': 2, 'Photos Count': 1, 'Type': None, 'Url': 'https

Processing rows: 1it [01:46, 106.35s/it]

{'Rating': 5, 'Text': None, 'Photos': [], 'Reviewer': {'Name': 'Enosh Leong', 'Reviews Count': 8, 'Photos Count': 3, 'Type': None, 'Url': 'https://www.google.com/maps/contrib/103556043045878733194/reviews?hl=en'}}
{'Rating': 5, 'Text': None, 'Photos': [], 'Reviewer': {'Name': 'angeline chee', 'Reviews Count': 7, 'Photos Count': None, 'Type': None, 'Url': 'https://www.google.com/maps/contrib/104372437490825760556/reviews?hl=en'}}
{'Rating': 4, 'Text': 'Food are delicious with affordable prices', 'Photos': ['https://lh5.googleusercontent.com/p/AF1QipNrv8qvwRooOPaCUk0iDAV6o1S9qnXFsKtETiHs=w600-h900-p-k-no', 'https://lh5.googleusercontent.com/p/AF1QipMA8mYNuNZl2Y1zT9sGeaTeOwDbr_DP4I85d-6N=w600-h900-p-k-no'], 'Reviewer': {'Name': 'ang xin', 'Reviews Count': 445, 'Photos Count': 1117, 'Type': 'Local Guide', 'Url': 'https://www.google.com/maps/contrib/109507639888235396365/reviews?hl=en'}}
{'Rating': 4, 'Text': 'The food and drinks are very good and the prices are reasonable. The only drawbac

Processing rows: 2it [05:16, 167.38s/it]

{'Rating': 5, 'Text': 'Nice food nice coffee nice place..\nQuite creative menu- coffee mixed with fruit. Next time will order and try. I love Thier avocado milk shake. Creamy and really zero sugar!', 'Photos': ['https://lh5.googleusercontent.com/p/AF1QipPKuXONKej4N_R0PtA9BrVnx-CRtj-oFlEqlo8t=w600-h450-p-k-no', 'https://lh5.googleusercontent.com/p/AF1QipMLXgc5kiEd1_dQeqzCUAwL3zW_VTBzIWt3i8Wd=w600-h450-p-k-no', 'https://lh5.googleusercontent.com/p/AF1QipN5r1hTksO6kyy7TyAgjPcpScK0jvrH1J67SjE6=w600-h450-p-k-no', 'https://lh5.googleusercontent.com/p/AF1QipO1Zr608OAgJPIDQmYd3L8I5ytcStvggPhMl-P9=w600-h450-p-k-no'], 'Reviewer': {'Name': 'Lisa Lim', 'Reviews Count': 7, 'Photos Count': 24, 'Type': None, 'Url': 'https://www.google.com/maps/contrib/115668183787176269778/reviews?hl=en'}}


Processing rows: 2it [05:21, 160.60s/it]


IndexError: list index out of range